# Parkinson's Disease Using Magnetic Resonance Imaging with Deep Learning Techniques



---



---





## Importing Modules

In [1]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt

## Preparing Dataset

In [2]:
height_ = 512
width_ = 512

In [3]:
# from google.colab import drive
# drive.mount('/content/drive/')

In [4]:
train = ImageDataGenerator(rescale=1/255, rotation_range = 0.2, brightness_range = (0.1,0.3))
test = ImageDataGenerator(rescale=1/255, rotation_range = 0.2, brightness_range = (0.1,0.3))

In [5]:
# cd drive/MyDrive

In [6]:
train_dataset = train.flow_from_directory("Train/Train/",
                                          target_size=(width_, height_), 
                                            # color_mode='grayscale',
                                            batch_size=8,
                                            class_mode='binary')
                                         
val_dataset = test.flow_from_directory("Test/Test/",
                                          target_size=(width_, height_),
                                            # color_mode='grayscale',
                                            batch_size=8,
                                            class_mode='binary')

Found 825 images belonging to 2 classes.
Found 166 images belonging to 2 classes.


In [7]:
train_dataset.class_indices

{'Normal': 0, 'Parkinson': 1}

## Preparing Model

with tf.device('/device:GPU:0'):
    SIZE_OF_IP = (width_, height_, 3) 
    model = keras.Sequential()

    # Convolutional layer and maxpool layer 1
    model.add(keras.layers.Conv2D(32,(3,3),activation='relu',input_shape=SIZE_OF_IP))
    model.add(keras.layers.MaxPool2D(2,2))

    # Convolutional layer and maxpool layer 2
    model.add(keras.layers.Conv2D(64,(3,3),activation='relu'))
    model.add(keras.layers.MaxPool2D(2,2))

    # Convolutional layer and maxpool layer 3
    model.add(keras.layers.Conv2D(128,(3,3),activation='relu'))
    model.add(keras.layers.MaxPool2D(2,2))
    model.add(keras.layers.Dropout(0.2))


    # # Convolutional layer and maxpool layer 4
    # model.add(keras.layers.Conv2D(128,(3,3),activation='relu'))
    # model.add(keras.layers.MaxPool2D(2,2))

    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(512,activation='relu'))

   # Op layer
    model.add(keras.layers.Dense(1,activation='sigmoid'))

In [8]:
from tensorflow.keras.layers import Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Convolution2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import TensorBoard,EarlyStopping

lesion_Classifier=Sequential()
lesion_Classifier.add(Convolution2D(16,(3,3),input_shape=(512,512,3),activation='relu'))
lesion_Classifier.add(MaxPooling2D(pool_size=(2,2)))
lesion_Classifier.add(Convolution2D(32,(3,3),activation='relu'))
lesion_Classifier.add(MaxPooling2D(pool_size=(2,2)))
lesion_Classifier.add(Convolution2D(64,(3,3),activation='relu'))
lesion_Classifier.add(MaxPooling2D(pool_size=(2,2)))
lesion_Classifier.add(Convolution2D(64,(3,3),activation='relu'))
lesion_Classifier.add(MaxPooling2D(pool_size=(2,2)))
lesion_Classifier.add(Convolution2D(128,(3,3),activation='relu'))
lesion_Classifier.add(MaxPooling2D(pool_size=(2,2)))
lesion_Classifier.add(Convolution2D(128,(3,3),activation='relu'))
lesion_Classifier.add(MaxPooling2D(pool_size=(2,2)))

lesion_Classifier.add(Flatten())
lesion_Classifier.add(Dense(512,activation='relu'))
lesion_Classifier.add(Dense(256,activation='relu'))
lesion_Classifier.add(Dense(1,activation='softmax'))

## Training

In [ ]:
with tf.device('/device:GPU:0'):
    lesion_Classifier.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
    es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)
    history=lesion_Classifier.fit(train_dataset,epochs=50,validation_data = val_dataset,callbacks=[es])

#     history = model.fit(train_dataset,epochs = 15,validation_data = val_dataset)

Epoch 1/50
104/104 [==============================] - 190s 2s/step - loss: 0.0000e+00 - accuracy: 0.4085 - val_loss: 0.0000e+00 - val_accuracy: 0.2651
Epoch 2/50
104/104 [==============================] - 179s 2s/step - loss: 0.0000e+00 - accuracy: 0.4085 - val_loss: 0.0000e+00 - val_accuracy: 0.2651
Epoch 3/50
104/104 [==============================] - 181s 2s/step - loss: 0.0000e+00 - accuracy: 0.4085 - val_loss: 0.0000e+00 - val_accuracy: 0.2651
Epoch 4/50
104/104 [==============================] - 181s 2s/step - loss: nan - accuracy: 0.4848 - val_loss: nan - val_accuracy: 0.7349
Epoch 5/50
104/104 [==============================] - 190s 2s/step - loss: nan - accuracy: 0.5915 - val_loss: nan - val_accuracy: 0.7349
Epoch 6/50
 69/104 [==================>...........] - ETA: 54s - loss: nan - accuracy: 0.5888

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
epochs = range(len(acc))

plt.plot(epochs, acc, 'b', label='Training Accuracy')
plt.plot(epochs, val_acc, 'r', label='Validation Accuracy')
plt.title('Accuracy Graph')
plt.legend()
plt.figure()


## Saving Model

#model.save("park_grayscale_drop_git.h5")

## Loading Model

In [ ]:
from keras import models

savedModel = models.load_model('park_grayscale_drop_git.h5')
savedModel.summary()

## Inferencing

In [ ]:
from PIL import Image, ImageOps

In [ ]:
def make_list_of_imgs(path):
    imgs = []
    valid_images = [".png"]
    for f in os.listdir(path):
        ext = os.path.splitext(f)[1]
        if ext.lower() not in valid_images:
            continue
        imgs.append(path + f)
    return imgs

In [ ]:
def tumor_prediction(*argv): 
    count_p = 0
    count_n = 0
    for arg in argv: 
        for filename in arg:
            # image_read = cv2.imread(filename)
            print(filename)
            im = Image.open(filename)
            im2 = im.convert('RGB')
            im2 = im2.resize((100, 100))
            # print(im1.shape)
            # image_read = cv2.resize(image_read, (height_, width_))
            image = np.array(im2) 
            # image = np.reshape(image, [100, 100, 3])
            plt.show()
            plt.imshow(image)
           
            X = np.expand_dims(image,axis=0)
            result = savedModel.predict(X)
#             print(val)
            if result == 1:
                plt.xlabel("Parkinsons",fontsize=15)
                count_p += 1


            elif result == 0:
                plt.xlabel("Normal",fontsize=15)
                count_n += 1
#         print("Normal: ", count_n)
#         print("Parkinsons: ", count_p)
    return count_p, count_n

### Testing on MRI images without Parkinson's Carcinoma

In [ ]:
fp, tn = tumor_prediction(make_list_of_imgs('Test/Test/Normal/') )
print(fp, tn)
# make_list_of_imgs('Test/Normal/')

### Testing with MRI images with Parkinson's carcinoma

In [ ]:
tp, fn = tumor_prediction(make_list_of_imgs('Test/Test/Parkinson/') )#classified
print(tp, fn)

In [ ]:
def accuracy_cnn(tp, tn, fp, fn):
    return (tp + tn)/(tp + tn + fp + fn)